# Práctica Web Scraping 2


1. Instalamos las dependencias 

    ```
    pip install requests beautifulsoup4
    ```

2. Luego importamos las dependecias necesarias

In [ ]:
import os # Librería para manejo de archivos y directorios
import requests # Librería para realizar solicitudes HTTP
from bs4 import BeautifulSoup # Librería para parsear HTML y extraer información
from urllib.parse import urljoin, urlparse # Librería para manejar URLs

3. Creamos una funcion para crear la carpeta en donde se guardaran las imagenes descargadas.

In [ ]:
def crear_carpeta(carpeta):
    if not os.path.exists(carpeta): # Verifica si la carpeta ya existe
        os.makedirs(carpeta) # Crea la carpeta si no existe

4. Creamos otra funcion que manejará las extenciones de la img ".png", "jpg", etc.

In [ ]:
def es_extension_valida(url):
    extensiones_validas = ['.png', '.jpg', '.jpeg', '.webp'] # Extensiones de imagen permitidas
    return any(url.lower().endswith(ext) for ext in extensiones_validas) # Verifica si la extensión de la imagen es válida

5. Creamos otra funcion para descargar las imagenes teniendo en cuenta la URL y luego guardarlo en la carpeta, en este caso "imagenes".

In [ ]:
def descargar_imagen(url, carpeta):
    try:
        respuesta = requests.get(url, stream=True)  # Realiza la solicitud HTTP para descargar la imagen
        respuesta.raise_for_status()  # Verifica si la solicitud fue exitosa
        nombre_imagen = os.path.join(carpeta, os.path.basename(urlparse(url).path))  # Obtiene el nombre de la imagen desde la URL
        with open(nombre_imagen, 'wb') as archivo:  # Abre un archivo en modo escritura binaria
            for chunk in respuesta.iter_content(chunk_size=8192):  # Descarga la imagen en trozos
                archivo.write(chunk)  # Escribe cada trozo en el archivo
        print(f"Imagen descargada: {nombre_imagen}")  # Imprime un mensaje confirmando la descarga
    except requests.exceptions.RequestException as e:  # Maneja errores de solicitud
        print(f"No se pudo descargar la imagen {url}: {e}")  # Imprime un mensaje de error en caso de que la imagen no se pueda descargar


6. Creamos otra funcion que será la principal, manejará la descarga de imagenes desde una URL epecifica y las guardara en una carpeta determinada.

In [ ]:
def scraper(url, carpeta='imagenes', limite=20):  # Para desactivar el límite, use "limite=None"
    # Función para descargar imágenes desde una URL dada, guardándolas en una carpeta especificada y limitando la cantidad de descargas.
    
    # Crea la carpeta si no existe
    crear_carpeta(carpeta)
    contador = 0  # Inicializa el contador de imágenes descargadas
    
    try:
        # Realiza la solicitud HTTP a la URL
        respuesta = requests.get(url)
        respuesta.raise_for_status()  # Verifica si la solicitud fue exitosa
        soup = BeautifulSoup(respuesta.content, 'html.parser')  # Parsea el contenido HTML con BeautifulSoup
        imagenes = soup.find_all('img')  # Encuentra todas las etiquetas <img> en el HTML
        
        for img in imagenes:
            if limite is not None and contador >= limite:  # Verifica si se ha alcanzado el límite de descargas
                break  # Si se alcanza el límite, termina el bucle
            src = img.get('src')  # Obtiene el atributo src de la etiqueta <img>
            if src:
                # Manejar URLs relativas
                src = urljoin(url, src)  # Convierte URLs relativas a absolutas
                if es_extension_valida(src):  # Verifica si la extensión de la imagen es válida
                    descargar_imagen(src, carpeta)  # Descarga la imagen
                    contador += 1  # Incrementa el contador de imágenes descargadas
                else:
                    print(f"Extensión no válida, se omite: {src}")  # Imprime un mensaje si la extensión no es válida
            else:
                print("Etiqueta <img> sin atributo src encontrado, se omite")  # Imprime un mensaje si la etiqueta <img> no tiene src
    except requests.exceptions.RequestException as e:  # Maneja errores de solicitud
        print(f"Error al conectar con la página {url}: {e}")  # Imprime un mensaje de error


7. Por ultimo, definimos la URL en la cual haremos scraping y llamamos a la funcion

In [ ]:
url = 'https://www.shutterstock.com/es/search/argentina-worldcup?PPC_GOO_LA_IG-621714502729=&cr=c&ds_ag=FF%3DDSA-All+Pages_AU%3DVisitors&ds_agid=58700008023276312&ds_cid=71700000100238870&ds_eid=700000001478290&gad_source=1&gclid=CjwKCAjw9cCyBhBzEiwAJTUWNQtUjvpOc3a0Auwn9UaGWajB5qEL1yXFfiz_9wgNohZe6ukdl3zqMhoCObcQAvD_BwE&gclsrc=aw.ds&kw=&utm_campaign=CO%3DLatam_LG%3DES_BU%3DIMG_AD%3DDSA_TS%3Dlggeneric_RG%3DAMER_AB%3DACQ_CH%3DSEM_OG%3DCONV_PB%3DGoogle&utm_medium=cpc&utm_source=GOOGLE'
scraper(url, limite=20) # Llama a la función scraper con la URL y el límite de descargas
                        # Para desactivar el limite, utilice "limite=None"